# 2025-05-01, Link Checker Report for HEB

In [18]:
# Import Packages
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [13]:
UNIT_FILTER = "HEB"  # Example: "MUS", "BEINECKE", or None

In [ ]:
# Connect to Database
conn = sqlite3.connect("/Users/if224/Documents/rds_git_repos/link_checker/link_checker.db")
print(f"Database connected")

# Overview

In [ ]:
# Summary Stats for Selected Unit

params = []
summary_query = """
SELECT 
    COUNT(*) AS Total_Links,
    SUM(CASE WHEN Status != '200' THEN 1 ELSE 0 END) AS Total_Errors
FROM (
    SELECT 
        bibid,
        url,
        Unit,
        Status,
        ROW_NUMBER() OVER (
            PARTITION BY bibid, url
            ORDER BY check_date DESC
        ) AS rn
    FROM bib_url_status_report
)
WHERE rn = 1 AND Unit = ?;
"""

summary_df = pd.read_sql_query(summary_query, conn, params=[UNIT_FILTER])


# Calculate error rate
totals = summary_df.iloc[0]
total_links = totals['Total_Links']
total_errors = totals['Total_Errors']
total_success = total_links - total_errors
error_rate = (total_errors / total_links) * 100 if total_links else 0

# Print summary
(f"""
### Summary for Unit: `{UNIT_FILTER or 'All Units'}`

- **Total Links Checked**: {total_links}
- **Total Errors**: {total_errors}
- **Successful Links**: {total_success}
- **Error Rate**: {error_rate:.2f}%
""")


### Summary for Unit: `HEB`

- Total Links Checked: 648
- Total Errors: 289
- Successful Links: 359
- Error Rate: 44.60%

In [ ]:
'\n### Summary for Unit: `HEB`\n\n- **Total Links Checked**: 648\n- **Total Errors**: 289\n- **Successful Links**: 359\n- **Error Rate**: 44.60%\n'

# Links and Errors by Domain

In [16]:
params = []
query = """
SELECT Domain, Status, COUNT(*) AS Error_Count
FROM bib_url_status_report
"""

if UNIT_FILTER:
    query += "WHERE Unit = ? AND Status != '200'\n"
    params.append(UNIT_FILTER)
else:
    query += "WHERE Status != '200'\n"

query += "GROUP BY Domain, Status ORDER BY Error_Count DESC"

# Query the data
domain_error_types = pd.read_sql_query(query, conn, params=params)
domain_error_types


,Domain,Status,Error_Count
0,www.yiddishbookcenter.org,"{""requests"": ""403"", ""selenium"": ""403""}",412
1,yizkor.nypl.org,"{""requests"": ""404"", ""selenium"": ""404""}",130
2,yizkor.nypl.org,404,67
3,www.archive.org,Exception: <class 'selenium.common.exceptions....,57
4,www.yiddishbookcenter.org,403,33
5,www.hebrewbooks.org,403,30
6,www.hebrewbooks.org,"{""requests"": ""403"", ""selenium"": ""403""}",24
7,web.nli.org.il,"{""requests"": ""403"", ""selenium"": ""403""}",8
8,beinecke.library.yale.edu,Exception: <class 'selenium.common.exceptions....,7
9,www.cbs.gov.il,Exception: <class 'selenium.common.exceptions....,7


In [22]:
# Format the output filename
today = datetime.today().strftime("%Y-%m-%d")
filename = f"{UNIT_FILTER}_linkerrors_{today}.csv"

# Build the query
# query = """
# SELECT *
# FROM bib_url_status_report
# WHERE Status != '200' AND Unit = ?
# """
query = """
SELECT 
    *
FROM (
    SELECT 
        ROW_NUMBER() OVER (
            PARTITION BY 
                bibID, URL, Unit
            ORDER BY 
                Check_Date DESC
        ) as rn,
        ID, 
        Unit, 
        BibID, 
        URL, 
        Status_Requests,
        Status_Selenium, 
        Error_Requests,
        Error_Selenium,
        Check_Date
    FROM 
        url_status_report
) 
WHERE 
    rn = 1 
AND 
    ( Status_Requests < 200 OR Status_Requests > 299 OR Status_Requests IS NULL )
AND 
    ( Status_Selenium < 200 OR Status_Selenium > 299 OR Status_Selenium IS NULL ) 
AND 
    Unit = ?
"""

# Run the query
error_links_df = pd.read_sql_query(query, conn, params=[UNIT_FILTER])

# Export to CSV
error_links_df.to_csv(filename, index=False,columns= [
    "Unit", 
    "BibID", 
    "URL", 
    "Status_Requests",
    "Status_Selenium", 
    "Error_Requests",
    "Error_Selenium",
    "Check_Date" 
    ])

# print(f"Saved error links to: {filename}")
error_links_df

,rn,ID,Unit,BibID,URL,Status_Requests,Status_Selenium,Error_Requests,Error_Selenium,Check_Date
0,1,257251,HEB,957687,http://www.yiddishbookcenter.org/books/browse,403.0,403,None,None,2025-04-16 16:12:36.711612
1,1,257252,HEB,1010822,http://www.yiddishbookcenter.org/books/browse,403.0,403,None,None,2025-04-16 16:12:36.711612
2,1,257253,HEB,1010824,http://www.yiddishbookcenter.org/books/browse,403.0,403,None,None,2025-04-16 16:12:36.711612
3,1,257254,HEB,1013941,http://www.yiddishbookcenter.org/books/browse,403.0,403,None,None,2025-04-16 16:12:36.711612
4,1,257255,HEB,1026522,http://www.yiddishbookcenter.org/books/browse,403.0,403,None,None,2025-04-16 16:12:36.711612
...,...,...,...,...,...,...,...,...,...,...
287,1,257539,HEB,16046097,https://www.yiddishbookcenter.org/collections/...,403.0,403,None,None,2025-04-16 16:12:36.711612
288,1,257540,HEB,16088777,http://www.yiddishbookcenter.org/collections/y...,403.0,403,None,None,2025-04-16 16:12:36.711612
289,1,257541,HEB,16212276,http://www.yiddishbookcenter.org/collections/y...,403.0,403,None,None,2025-04-16 16:12:36.711612
290,1,257542,HEB,16397286,https://www.yiddishbookcenter.org/collections/...,403.0,403,None,None,2025-04-16 16:12:36.711612
